Чтение и преобразование данных:

In [74]:
from math import sqrt
import numpy as np
import pandas
import csv
%matplotlib inline

def read_file(filename):
    with open(filename, 'r') as file:
        lines = list(csv.reader(file))
    return lines

fires = read_file('forestfires.csv')

for fire in fires:
    # удаляем дату
    fire.pop(3)
    # добавляем столбец свободных членов
    fire.append(1.0)

# убираем строку с названиями столбцов
headings = fires[0]
fires = fires[1:]
# изменяем столбец с названиями месяцов
# на индикатор летнего сезона
for i in range(len(fires)):
    fires[i][2] = 1.0 if fires[i][2] in ['jun', 'jul', 'aug'] else 0.0
    fires[i] = [float(number) for number in fires[i]]

# перемешиваем выборку
random.shuffle(fires)

Функции:
* работаем с преобразованной функцией
* работаем с исходными значениями, применив обратное преобразование
* считаем среднеквадратичную ошибку

In [75]:
def change(c):
    # разбиваем выборку на две части
    learn = fires[:(len(fires) * 7 // 10)]
    # столбец [:-2] - "area"
    z_learn = np.matrix([fire[:-2] + fire[-1:] for fire in learn])
    # преобразуем данные и транспонируем матрицу
    x_learn = np.matrix([np.log(c + fire[-2]) for fire in learn]).transpose()
    # считаем для нее обратную ..., получаем параметр
    theta_ls = np.linalg.inv(
        z_learn.transpose() * z_learn) * z_learn.transpose() * x_learn
    
    # то же самое для тестовой выборки
    test = fires[(len(fires) * 7 // 10):]
    z_test = np.matrix([fire[:-2] + fire[-1:] for fire in test])
    x_test = np.matrix([np.log(c + fire[-2]) for fire in test]).transpose()
    result = z_test * theta_ls
    
    # считаем среднеквадратичную ошибку
    return sqrt(np.mean(np.array(
            (np.log(c +  result) - np.log(c + x_test)).transpose()[0]) ** 2))

# аналогично
def reverse(c):
    learn = fires[:(len(fires) * 7 // 10)]
    z_learn = np.matrix([fire[:-2] + fire[-1:] for fire in learn]) # [-2] - area
    x_learn = np.matrix([np.log(c + fire[-2]) for fire in learn]).transpose()
    theta_ls = np.linalg.inv(
        z_learn.transpose() * z_learn) * z_learn.transpose() * x_learn
    
    test = fires[(len(fires) * 7 // 10):]
    z_test = np.matrix([fire[:-2] + fire[-1:] for fire in test])
    x_test = np.matrix([np.log(c + fire[-2]) for fire in test]).transpose()
    result = z_test * theta_ls

    return sqrt(np.mean(np.array(
            ((np.exp(result) - c) - (np.exp(x_test) - c)).transpose()[0]) ** 2))

for c in range(1, 100, 10):
    print("Преобразованных ln(" + str(c) + " + x):", "%.4f" % change(c))
    print("Исходных к ln(" + str(c) + " + x):", "%.4f" % reverse(c))
    print("\n")

Преобразованных ln(1 + x): 0.6581
Исходных к ln(1 + x): 27.2382


Преобразованных ln(11 + x): 0.0446
Исходных к ln(11 + x): 26.9232


Преобразованных ln(21 + x): 0.0191
Исходных к ln(21 + x): 26.9733


Преобразованных ln(31 + x): 0.0110
Исходных к ln(31 + x): 27.0646


Преобразованных ln(41 + x): 0.0073
Исходных к ln(41 + x): 27.1655


Преобразованных ln(51 + x): 0.0052
Исходных к ln(51 + x): 27.2674


Преобразованных ln(61 + x): 0.0040
Исходных к ln(61 + x): 27.3674


Преобразованных ln(71 + x): 0.0031
Исходных к ln(71 + x): 27.4643


Преобразованных ln(81 + x): 0.0025
Исходных к ln(81 + x): 27.5577


Преобразованных ln(91 + x): 0.0021
Исходных к ln(91 + x): 27.6476




В любом случае ошибка очень велика. Найдем константу, при котором достигается минимальное среднеквадратичное отклонение.

In [76]:
# строим массив из отклонений при различных параметрах
find_const = [normal(c) for c in np.arange(0.1, 100, 0.1)]
# находим его минимум
best_const = np.argmin(find_const) / 10 - 1
print("c = " + str(best_const))

c = 98.8


Проверим зависимость от способа разбиения выборки:

In [77]:
for i in range(10):
    # перемешиваем выборку
    random.shuffle(fires)
    # считаем отклонение
    print(normal(best_const))

2.33081012605e-83
1.20161785931e-82
1.23520450286e-82
6.35909325867e-83
6.23391398674e-83
1.22452985049e-82
1.90146175187e-82
2.9473440493e-83
7.21033937855e-83
7.72485701105e-83


Среднеквадратичное отклонение очень сильно зависит от способа разбиения. Так что линейная регрессия работает весьма плохо.